* TensorFlow: An Introduction *
====================
![MindLABLogo](images/mindlab-logo-simple.png "MindLab") 
![UNLogo](images/unal-logo.png "MindLab") 


** This notebook is heavily based on the TensorFlow documentation, particularly: **
    
   * https://www.tensorflow.org/versions/master/get_started/basic_usage.html 
   * https://www.tensorflow.org/versions/master/tutorials/seq2seq/index.html 
   * https://www.tensorflow.org/api_docs
   * https://www.tensorflow.org/tutorials

<h3>To use TensorFlow we need to understand how TensorFlow:</h3>
* Represents computations as graphs (Theano alike)
* Executes graphs in the context of Sessions
* Represents data as tensors
* Maintains state with Variables
* Uses feeds and fetches to get data into and out of arbitrary operations

Representing computations as graphs
==========================

* Simmilarly as happens with Theano, TensorFlow is built with the concept of variable dependence through a graph. 
* TensorFlow programs are usually structured into a construction phase, that assembles a graph, and an execution phase that uses a session to execute ops in the graph.
* TensorFlow can be used from C, C++, and Python programs. It is presently much easier to use the Python library to assemble graphs, as it provides a large set of helper functions not available in the C and C++ libraries.
* The session libraries have equivalent functionalities for the three languages.

In [2]:
import tensorflow as tf
import numpy as np
# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)



The default graph now has three nodes: two constant() ops and one matmul() op. To actually multiply the matrices, and get the result of the multiplication, we must launch the graph in a session.

Launching the graph in a session
===================

A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.

Launching follows construction. To launch a graph, create a Session object. Without arguments the session constructor launches the default graph.

See the  [Session class](https://www.tensorflow.org/versions/master/api_docs/python/client.html#session-management "Session")  for the complete session API.

In [20]:
#Allocating just the GPU memory that we need
#http://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# Launch the default graph.
sess = tf.Session()

# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of threes ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> If everithing goes well we should see [[ 12.]]

# Close the Session when we're done.
sess.close()

[[ 12.]]


For ease of use in interactive Python environments, such as IPython you can instead use the InteractiveSession class, and the Tensor.eval() and Operation.run() methods. This avoids having to keep a variable holding the session.

The only difference with a regular Session is that an InteractiveSession installs itself as the default session on construction. The methods Tensor.eval() and Operation.run() will use that session to run ops.

This is convenient in interactive shells and IPython notebooks, as it avoids having to pass an explicit Session object to run ops.



In [21]:
# Enter an interactive TensorFlow Session.
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# Initialize 'x' using the run() method of its initializer op.
x.initializer.run()

# Add an op to subtract 'a' from 'x'.  Run it and print the result
sub = tf.sub(x, a)
print(sub.eval())
# ==> [-2. -1.]

# Close the Session when we're done.
sess.close()

[-2. -1.]


Tensors and Variables
===============================

* *TensorFlow* programs use a <b>Tensor</b> data structure to represent all data -- only tensors are passed between operations in the computation graph. You can think of a TensorFlow tensor as an n-dimensional array or list. A tensor has a static type, a rank, and a shape.



* A Variable is a value that lives in TensorFlow's computation graph. It can be used and even modified by the computation. In machine learning applications, one generally has the model parameters be Variables.

<b>Variables</b> maintain state across executions of the graph. The following example shows a variable serving as a simple counter. See the <a href="https://www.tensorflow.org/versions/master/api_docs/python/state_ops.html#variables-2">Variable</a> class for more details.




In [22]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))


0
1
2
3


Fetches and Feeds 
==============================


* The examples above introduce tensors into the computation graph by storing them in Constants and Variables. TensorFlow also provides a feed and fetch mechanisms for patching/retrieving a tensor directly into any operation in the graph.

* A feed temporarily replaces the output of an operation with a tensor value. **You supply feed data as an argument to a run() call**. The feed is only used for the run call to which it is passed. The most common use case involves designating specific operations to be "feed" operations by using tf.placeholder() to create them
* To fetch the outputs of operations, execute the graph with a run() call on the Session object and **pass in the tensors to retrieve**. In the previous example we fetched the single node state, but you can also fetch multiple tensors:



In [23]:
#Let's fetch something
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

# output:
# [array([ 21.], dtype=float32), array([ 7.], dtype=float32)]


[21.0, 7.0]


In [25]:
#Lets feed something
#A feed temporarily replaces the output of an operation with a tensor value. You supply feed data as an argument to a run() call. 
#The feed is only used for the run call to which it is passed. The most common use case involves designating specific operations to be "feed" operations by using tf.placeholder() to create them:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

# output:
# [array([ 14.], dtype=float32)]



[array([ 14.], dtype=float32)]


Basic Usage example: Fitting a line 
===================

Taken from: https://www.tensorflow.org/versions/master/get_started/index.html

In [34]:
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3


In [35]:
# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b



In [36]:
# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)


In [37]:
# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in xrange(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))


0 [-0.00192705] [ 0.48994273]
20 [ 0.05797241] [ 0.32253236]
40 [ 0.08883978] [ 0.30598336]
60 [ 0.09703647] [ 0.30158886]
80 [ 0.09921306] [ 0.30042192]
100 [ 0.09979101] [ 0.30011207]
120 [ 0.0999445] [ 0.30002975]
140 [ 0.09998529] [ 0.30000791]
160 [ 0.09999609] [ 0.3000021]
180 [ 0.09999897] [ 0.30000058]
200 [ 0.09999974] [ 0.30000016]


Using Multiple GPU's
========================

If you would like to run TensorFlow on multiple GPUs, you can construct your model in a multi-tower fashion where each tower is assigned to a different GPU. For example:


In [5]:

# Creates a graph.
c = []
for d in ['/gpu:0', '/cpu:0']: # It was /gpu:2 but in HAL we have just 1 Titan X
  with tf.device(d):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
  sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print sess.run(sum)


[[  44.   56.]
 [  98.  128.]]


Sequence-to-Sequence Models in TensorFlow
==============================

Recurrent neural networks can learn to model language, as already discussed in the RNN examples (there is also <a href="https://www.tensorflow.org/versions/master/tutorials/recurrent/index.html">this one</a> of RNN in tensorflow). This raises an interesting question: could we condition the generated words on some input and generate a meaningful response? For example, could we train a neural network to translate from English to French? It turns out that the answer is yes.



With this example we will be able to build and train such a system end-to-end. We are assuming you have already installed via the pip package, have cloned the tensorflow git repository, and are in the root of the git tree.



This tutorial references the following files from the models/rnn folder in tensorflow instalation path (mine is on /home/jsotaloram/nlp/tensorflow/tensorflow/models/rnn/)
<table>
    <tr>
        <td>**File**</td> <td>**What's in it?**</td>
<tr>
        <td>seq2seq.py</td> <td>Library for building sequence-to-sequence models.</td>

</tr>
<tr>
        <td>translate/data_utils.py</td> <td>Helper functions for preparing translation data.</td>

</tr>

<tr>
        <td>translate/seq2seq_model.py</td> <td>Neural translation sequence-to-sequence model.</td>

</tr>
<tr>
        <td>translate/translate.py</td> <td>Binary that trains and runs the translation model.</td>

</tr>

</tr>
    
</table>
<a href="https://www.tensorflow.org/versions/master/tutorials/seq2seq/index.html"> Complete tutorial </a><br>


In [1]:
import sys
sys.path.insert(1,'/home/jsotaloram/nlp/tensorflow/tensorflow/models/rnn/translate/')
import seq2seq_model
import data_utils

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time

import tensorflow.python.platform

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin                                                          
import tensorflow as tf

from tensorflow.python.platform import gfile



In [2]:
#Now let's define the flags for our parameters
tf.app.flags.DEFINE_float("learning_rate", 0.5, "Learning rate.")
tf.app.flags.DEFINE_float("learning_rate_decay_factor", 0.99,
                          "Learning rate decays by this much.")
tf.app.flags.DEFINE_float("max_gradient_norm", 5.0,
                          "Clip gradients to this norm.")
tf.app.flags.DEFINE_integer("batch_size", 64,
                            "Batch size to use during training.")
tf.app.flags.DEFINE_integer("size", 1024, "Size of each model layer.")
tf.app.flags.DEFINE_integer("num_layers", 3, "Number of layers in the model.")
tf.app.flags.DEFINE_integer("en_vocab_size", 40000, "English vocabulary size.")
tf.app.flags.DEFINE_integer("fr_vocab_size", 40000, "French vocabulary size.")
tf.app.flags.DEFINE_string("data_dir", "/home/datasets/datasets1/wmt_dataset/", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "/home/jsotaloram/nlp/notebook/pretrained_model", "Training directory.")
tf.app.flags.DEFINE_integer("max_train_data_size", 0,
                            "Limit on the size of training data (0: no limit).")
tf.app.flags.DEFINE_integer("steps_per_checkpoint", 100,
                            "How many training steps to do per checkpoint.")
tf.app.flags.DEFINE_boolean("decode", True,
                            "Set to True for interactive decoding.")
tf.app.flags.DEFINE_boolean("self_test", False,
                            "Run a self-test if this is set to True.")
FLAGS = tf.app.flags.FLAGS


In [3]:
# We use a number of buckets and pad to the closest one for efficiency.                                                    
# See seq2seq_model.Seq2SeqModel for details of how they work.                                                             
_buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]


In [5]:
def read_data(source_path, target_path, max_size=None):
  """Read data from source and target files and put into buckets.
  Args:
    source_path: path to the files with token-ids for the source language.
    target_path: path to the file with token-ids for the target language;
      it must be aligned with the source file: n-th line contains the desired
      output for n-th line from the source_path.
    max_size: maximum number of lines to read, all other will be ignored;
      if 0 or None, data files will be read completely (no limit).
  Returns:
    data_set: a list of length len(_buckets); data_set[n] contains a list of
      (source, target) pairs read from the provided data files that fit
      into the n-th bucket, i.e., such that len(source) < _buckets[n][0] and
      len(target) < _buckets[n][1]; source and target are lists of token-ids.
  """
  data_set = [[] for _ in _buckets]
  with gfile.GFile(source_path, mode="r") as source_file:
    with gfile.GFile(target_path, mode="r") as target_file:
      source, target = source_file.readline(), target_file.readline()
      counter = 0
      while source and target and (not max_size or counter < max_size):
        counter += 1
        if counter % 100000 == 0:
          print("  reading data line %d" % counter)
          sys.stdout.flush()
        source_ids = [int(x) for x in source.split()]
        target_ids = [int(x) for x in target.split()]
        target_ids.append(data_utils.EOS_ID)
        for bucket_id, (source_size, target_size) in enumerate(_buckets):
          if len(source_ids) < source_size and len(target_ids) < target_size:
            data_set[bucket_id].append([source_ids, target_ids])
            break
        source, target = source_file.readline(), target_file.readline()
  return data_set


In [6]:
def create_model(session, forward_only):
  """Create translation model and initialize or load parameters in session."""
  model = seq2seq_model.Seq2SeqModel(
      FLAGS.en_vocab_size, FLAGS.fr_vocab_size, _buckets,
      FLAGS.size, FLAGS.num_layers, FLAGS.max_gradient_norm, FLAGS.batch_size,
      FLAGS.learning_rate, FLAGS.learning_rate_decay_factor,
      forward_only=forward_only)
  ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
  if ckpt and gfile.Exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(session, ckpt.model_checkpoint_path)
  else:
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
  return model


In [14]:
def train():
  """Train a en->fr translation model using WMT data."""
  # Prepare WMT data.
  print("Preparing WMT data in %s" % FLAGS.data_dir)
  en_train, fr_train, en_dev, fr_dev, _, _ = data_utils.prepare_wmt_data(
      FLAGS.data_dir, FLAGS.en_vocab_size, FLAGS.fr_vocab_size)
  with tf.Session() as sess:
    # Create model.
    print("Creating %d layers of %d units." % (FLAGS.num_layers, FLAGS.size))
    model = create_model(sess, False)
    # Read data into buckets and compute their sizes.
    print ("Reading development and training data (limit: %d)."
           % FLAGS.max_train_data_size)
    dev_set = read_data(en_dev, fr_dev)
    train_set = read_data(en_train, fr_train, FLAGS.max_train_data_size)
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(_buckets))]
    train_total_size = float(sum(train_bucket_sizes))
    # A bucket scale is a list of increasing numbers from 0 to 1 that we'll use
    # to select a bucket. Length of [scale[i], scale[i+1]] is proportional to
    # the size if i-th training bucket, as used later.
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]
    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    while True:
      # Choose a bucket according to data distribution. We pick a random number
      # in [0, 1] and use the corresponding interval in train_buckets_scale.
      random_number_01 = np.random.random_sample()
      bucket_id = min([i for i in xrange(len(train_buckets_scale))
                       if train_buckets_scale[i] > random_number_01])
      # Get a batch and make a step.
      start_time = time.time()
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          train_set, bucket_id)
      _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                   target_weights, bucket_id, False)
      step_time += (time.time() - start_time) / FLAGS.steps_per_checkpoint
      loss += step_loss / FLAGS.steps_per_checkpoint
      current_step += 1
      # Once in a while, we save checkpoint, print statistics, and run evals.
      if current_step % FLAGS.steps_per_checkpoint == 0:
        # Print statistics for the previous epoch.
        perplexity = math.exp(loss) if loss < 300 else float('inf')
        print ("global step %d learning rate %.4f step-time %.2f perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
          sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)
        # Save checkpoint and zero timer and loss.
        checkpoint_path = os.path.join(FLAGS.train_dir, "translate.ckpt")
        model.saver.save(sess, checkpoint_path, global_step=model.global_step)
        step_time, loss = 0.0, 0.0
        # Run evals on development set and print their perplexity.
        for bucket_id in xrange(len(_buckets)):
          encoder_inputs, decoder_inputs, target_weights = model.get_batch(
              dev_set, bucket_id)
          _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
          eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
          print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))
        sys.stdout.flush()


In [9]:
def decode():
  with tf.Session() as sess:
    # Create model and load parameters.
    model = create_model(sess, True)
    model.batch_size = 1  # We decode one sentence at a time.
    # Load vocabularies.
    en_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.en" % FLAGS.en_vocab_size)
    fr_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.fr" % FLAGS.fr_vocab_size)
    en_vocab, _ = data_utils.initialize_vocabulary(en_vocab_path)
    _, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)
    # Decode from standard input.
    sys.stdout.write("> ")
    sys.stdout.flush()
    sentence = sys.stdin.readline()
    while sentence:
      # Get token-ids for the input sentence.
      token_ids = data_utils.sentence_to_token_ids(sentence, en_vocab)
      # Which bucket does it belong to?
      bucket_id = min([b for b in xrange(len(_buckets))
                       if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)
      # Get output logits for the sentence.
      _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
      # This is a greedy decoder - outputs are just argmaxes of output_logits.
      outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
      # If there is an EOS symbol in outputs, cut them at that point.
      if data_utils.EOS_ID in outputs:
        outputs = outputs[:outputs.index(data_utils.EOS_ID)]
      # Print out French sentence corresponding to outputs.
      print(" ".join([rev_fr_vocab[output] for output in outputs]))
      print("> ", end="")
      sys.stdout.flush()
      sentence = sys.stdin.readline()


In [10]:
def self_test():
  """Test the translation model."""
  with tf.Session() as sess:
    print("Self-test for neural translation model.")
    # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
    model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)
    sess.run(tf.initialize_all_variables())
    # Fake data set for both the (3, 3) and (6, 6) bucket.
    data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
    for _ in xrange(5):  # Train the fake model for 5 steps.
      bucket_id = random.choice([0, 1])
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          data_set, bucket_id)
      model.step(sess, encoder_inputs, decoder_inputs, target_weights,
                 bucket_id, False)


In [11]:
self_test()

Self-test for neural translation model.


In [19]:
def main(_):
  if FLAGS.self_test:
    self_test()
  elif FLAGS.decode:
    decode()
  else:
    train()

In [3]:
cd /home/jsotaloram/nlp/tensorflow/tensorflow/models/rnn/translate/


/home/jsotaloram/nlp/tensorflow/tensorflow/models/rnn/translate


In [2]:
%run translate.py --data_dir /home/datasets/datasets1/wmt_dataset/ --train_dir /home/jsotaloram/nlp/notebook/pretrained_model --en_vocab_size=40000 --fr_vocab_size=40000


Preparing WMT data in /home/datasets/datasets1/wmt_dataset/
Creating 3 layers of 1024 units.
Created model with fresh parameters.
Reading development and training data (limit: 0).
  reading data line 100000
  reading data line 200000
  reading data line 300000
  reading data line 400000
  reading data line 500000
  reading data line 600000
  reading data line 700000
  reading data line 800000
  reading data line 900000
  reading data line 1000000
  reading data line 1100000
  reading data line 1200000
  reading data line 1300000
  reading data line 1400000
  reading data line 1500000
  reading data line 1600000
  reading data line 1700000
  reading data line 1800000
  reading data line 1900000
  reading data line 2000000
  reading data line 2100000
  reading data line 2200000
  reading data line 2300000
  reading data line 2400000
  reading data line 2500000
  reading data line 2600000
  reading data line 2700000
  reading data line 2800000
  reading data line 2900000
  reading data li

ResourceExhaustedError: OOM when allocating tensor with shapedim { size: 2048 } dim { size: 2048 }
	 [[Node: gradients_3/model_with_buckets/embedding_attention_seq2seq_3/embedding_attention_decoder/attention_decoder/MultiRNNCell_23/Cell0/GRUCell/Gates/Linear/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](model_with_buckets/embedding_attention_seq2seq_3/embedding_attention_decoder/attention_decoder/MultiRNNCell_23/Cell0/GRUCell/Gates/Linear/concat, gradients_3/model_with_buckets/embedding_attention_seq2seq_3/embedding_attention_decoder/attention_decoder/MultiRNNCell_23/Cell0/GRUCell/Gates/add_grad/Reshape)]]
	 [[Node: clip_by_global_norm_3/clip_by_global_norm_3/_15/_2999 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_200736_clip_by_global_norm_3/clip_by_global_norm_3/_15",

In [4]:
%run translate.py --data_dir /home/datasets/datasets1/wmt_dataset/ --train_dir /home/jsotaloram/nlp/notebook/pretrained_model --en_vocab_size=40000 --fr_vocab_size=40000


Preparing WMT data in /home/datasets/datasets1/wmt_dataset/
Creating 3 layers of 1024 units.
Created model with fresh parameters.
Reading development and training data (limit: 1000000).
  reading data line 100000
  reading data line 200000
  reading data line 300000
  reading data line 400000
  reading data line 500000
  reading data line 600000
  reading data line 700000
  reading data line 800000
  reading data line 900000
  reading data line 1000000


ResourceExhaustedError: OOM when allocating tensor with shapedim { size: 64 } dim { size: 2048 }
	 [[Node: model_with_buckets/embedding_attention_seq2seq_3/embedding_attention_decoder/attention_decoder/MultiRNNCell_45/Cell1/GRUCell/Gates/Linear/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](model_with_buckets/embedding_attention_seq2seq_3/embedding_attention_decoder/attention_decoder/MultiRNNCell_45/Cell1/GRUCell/Gates/Linear/concat, embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/MultiRNNCell/Cell1/GRUCell/Gates/Linear/Matrix)]]
	 [[Node: global_norm_3/global_norm/_4563 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_227919_global_norm_3/global_norm", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'model_with_buckets/

TensorBoard: Visualizing Learning
===================================

TensorFlow programs, we've included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it. 
We'll run the MNIST tutorial with modifications to see some metrics on tensorboard

In [1]:
cd /home/jsotaloram/nlp/tensorflow/tensorflow/examples/tutorials/mnist/

/home/jsotaloram/nlp/tensorflow/tensorflow/examples/tutorials/mnist


In [7]:
import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


In [11]:
#All the model is this line
y = tf.nn.softmax(tf.matmul(x, W) + b)


In [13]:
#To implement cross-entropy we need to first add a new placeholder to input the correct answers:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


In [14]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


In [15]:
init = tf.initialize_all_variables()


In [16]:
sess = tf.Session()
sess.run(init)


In [17]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


In [18]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))


In [19]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [21]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9098


In [ ]:
#Now lets modify the code to put some summaries in tensorboard

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.python.platform
import input_data
import tensorflow as tf


In [3]:

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')
flags.DEFINE_integer('max_steps', 1000, 'Number of steps to run trainer.')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')



In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
sess = tf.InteractiveSession()

# Create the model
x = tf.placeholder('float', [None, 784], name='x-input')
W = tf.Variable(tf.zeros([784, 10]), name='weights')
b = tf.Variable(tf.zeros([10], name='bias'))




In [8]:
# use a name scope to organize nodes in the graph visualizer
with tf.name_scope('Wx_b'):
  y = tf.nn.softmax(tf.matmul(x, W) + b)


In [9]:

# Add summary ops to collect data
_ = tf.histogram_summary('weights', W)
_ = tf.histogram_summary('biases', b)
_ = tf.histogram_summary('y', y)


In [14]:
# Define loss and optimizer
y_ = tf.placeholder('float', [None, 10], name='y-input')
# More name scopes will clean up the graph representation
with tf.name_scope('xent'):
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
    _ = tf.scalar_summary('cross entropy', cross_entropy)

with tf.name_scope('test'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    _ = tf.scalar_summary('accuracy', accuracy)


In [16]:
# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter('/tmp/mnist_logs', sess.graph_def)
tf.initialize_all_variables().run()


In [17]:
  # Train the model, and feed in test data and record summaries every 10 steps
for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summary data, and the accuracy
        if FLAGS.fake_data:
            batch_xs, batch_ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
            feed = {x: batch_xs, y_: batch_ys}
        else:
            feed = {x: mnist.test.images, y_: mnist.test.labels}
        result = sess.run([merged, accuracy], feed_dict=feed)
        summary_str = result[0]
        acc = result[1]
        writer.add_summary(summary_str, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:
        batch_xs, batch_ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
        feed = {x: batch_xs, y_: batch_ys}
        sess.run(train_step, feed_dict=feed)


usage: __main__.py [-h] [--fake_data FAKE_DATA] [--nofake_data]
                   [--max_steps MAX_STEPS] [--learning_rate LEARNING_RATE]
__main__.py: error: unrecognized arguments: -f /home/jsotaloram/.ipython/profile_default/security/kernel-1783fdc4-3ea1-43ad-adee-0abf8d6d4ad5.json --profile-dir /home/jsotaloram/.ipython/profile_default


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [19]:
#The command to run tensorflow is 
%run /home/jsotaloram/nlp/tensorflow/tensorflow/tensorboard/tensorboard.py --logdir=/home/jsotaloram/nlp/

AttributeError: 'NoneType' object has no attribute 'strip'